
# Conflicts and versions 

Conflicts are an essential part of any distributed system. 
Conflicts arise when there are two or more valid versions of the same message.

As we've seen from the previous notebooks, the network can be unreliable: there may be longer delays. But what about faults/conflicts from the clients/message producers? Moreover, a message might have multiple valid versions.  It could be that peers received two versions of the message, both being signed and valid. What version to choose? What to do when an order of the message is important? 



In [180]:
# Initialize the experiment:
import networkx as nx
import p2psimpy as p2p
import warnings
warnings.filterwarnings('ignore')

# Load the previous experiment configurations
exper = p2p.BaseSimulation.load_experiment(expr_dir='crash_gossip')
#exper = p2p.BaseSimulation.load_experiment(expr_dir='gossip_expr')
#exper = p2p.BaseSimulation.load_experiment(expr_dir='gossip_exprNew')

Locations, topology, peer_services, serv_impl = exper


## Client generating conflicting information

Let's assign first adversary nodes, we will assign randomly: 

In [181]:
import random
import string

from p2psimpy import BaseMessage, GossipMessage, MessageProducer, PullGossipService

from p2psimpy.consts import TEMPERED
from p2psimpy.config import Config, Func, Dist

class Transaction(BaseMessage):
    pass 

class ConflictMessageProducer(MessageProducer):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.balance = 100 
        
        
    def _generate_tx(self):        
        msg_phash = ''.join(random.choices(string.ascii_uppercase, k=10))
        diff = random.randint(1, 9)        
        data = {'hash': msg_phash, 'balance': self.balance-diff, 'diff': diff}
        tx = Transaction(self.peer, data)
        
        msg_id = '_'.join((str(self.peer.peer_id), str(self.counter)))
        msg_ttl = self.init_ttl
        return GossipMessage(self.peer, msg_id, tx, msg_ttl, 
            pre_task=self.pre_task, post_task=self.post_task)
    
    def produce_transaction(self):
        # generate new transaction
        msg = self._generate_tx()
        if self.balance - msg.data.data['diff'] < 0:
            return
        
        cons = list(self.peer.connections.keys())
        m_ix = len(cons) // 2
        for p in cons[:m_ix]:
            self.peer.send(p, msg)
            
        # Generate conflicting message as if previous transaction hasn't happened  
        msg = self._generate_tx()
        for p in cons[m_ix:]:
            self.peer.send(p, msg)
        
        self.peer.store('msg_time', str(self.peer.peer_id) +'_' + str(self.counter), self.peer.env.now)
        self.peer.store('msg_data', str(self.peer.peer_id) + '_' + str(self.counter), msg)
        self.balance -= msg.data.data['diff']
        self.counter+=1

def validate_task(msg, peer):
    # time it takes to verify the signature
    crypto_verify = Dist('norm', (1, 0.2)) 
    # time to verify the message data
    msg_verify = Dist("lognorm", (0.49512563, 4.892564, 0.0425785)) 
    
    yield peer.env.timeout(crypto_verify.get() + msg_verify.get())
    if isinstance(msg, Transaction):        
        tx = msg.data
        if tx == TEMPERED or tx['balance'] < 0:
            # You can decide what to do in this case.
            return False
    return True

class MsgConfig(Config):
    pre_task = Func(validate_task)

peer_services['client'].service_map['MessageProducer'] = MsgConfig
serv_impl['MessageProducer'] = ConflictMessageProducer
serv_impl['RangedPullGossipService'] = PullGossipService


## Run simulation 

Let's see how adversarial agents together with crashing nodes affect the message dissemination. 

In [182]:
peer_services

{'client': PeerType(config=<class 'p2psimpy.config.PeerConfig'>, service_map={'BaseConnectionManager': None, 'MessageProducer': <class '__main__.MsgConfig'>}),
 'peer': PeerType(config=<class 'p2psimpy.config.PeerConfig'>, service_map={'BaseConnectionManager': None, 'RandomDowntime': <class 'p2psimpy.config.DowntimeConfig'>, 'RangedPullGossipService': <class 'p2psimpy.config.GossipConfig'>})}

In [183]:
# Init Graph
sim = p2p.BaseSimulation(Locations, topology, peer_services, serv_impl)
sim.run(5_200)

## Message analysis

Let's see how this fraction of adverserial nodes affected the network. 


In [184]:
import pandas as pd

def message_data(sim, peer_id, storage_name):
    store = sim.peers[peer_id].storage[storage_name].txs
    for msg_id, tx in store.items():
        client_id, msg_num = msg_id.split('_')
        client_tx = sim.peers[int(client_id)].storage[storage_name].txs[msg_id]
        yield (int(msg_num), tx.data == client_tx.data)
        
def get_gossip_table(sim, storage_name, func):
    return pd.DataFrame({k: dict(func(sim, k, storage_name)) 
                         for k in set(sim.types_peers['peer'])}).sort_index()

    
df = get_gossip_table(sim, 'msg_data', message_data)
df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False
2,True,False,False,False,True,True,True,True,False,True,True,True,False,False,True
3,True,True,False,False,False,True,True,True,False,True,True,False,False,True,True
4,True,True,False,False,True,False,False,True,True,True,False,False,True,False,False
5,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False
6,False,False,False,True,False,False,False,True,False,False,True,False,False,False,True
7,True,True,False,True,True,False,False,False,True,False,True,False,True,True,True
8,True,True,True,True,True,True,True,True,True,True,False,False,True,True,False
9,True,True,True,True,True,False,True,False,True,True,True,True,True,False,True
10,True,False,False,False,False,False,False,True,False,True,False,False,False,True,True


In [185]:
df[df==False].count()

1      4
2      9
3     12
4      9
5     14
6     11
7      7
8      5
9     11
10     4
11     7
12    13
13     7
14    11
15     4
dtype: int64

In [186]:
sim.peers[6].storage['msg_data'].txs

{'16_1': GossipMessage:Transaction:{'hash': 'VPAYFUJZQP', 'balance': 92, 'diff': 8},
 '17_2': GossipMessage:Transaction:{'hash': 'NPIBKDUIJF', 'balance': 90, 'diff': 9},
 '16_2': GossipMessage:Transaction:{'hash': 'YUGMUCUBPA', 'balance': 83, 'diff': 9},
 '16_4': GossipMessage:Transaction:{'hash': 'VJOFQYTIZY', 'balance': 76, 'diff': 2},
 '17_1': GossipMessage:Transaction:{'hash': 'QGHTQOOZXC', 'balance': 94, 'diff': 6},
 '17_4': GossipMessage:Transaction:{'hash': 'CBFTASBIUH', 'balance': 84, 'diff': 5},
 '17_3': GossipMessage:Transaction:{'hash': 'CWTXUKBSHY', 'balance': 88, 'diff': 2},
 '16_3': GossipMessage:Transaction:{'hash': 'SCKPYFGFHN', 'balance': 78, 'diff': 5},
 '16_7': GossipMessage:Transaction:{'hash': 'RFAVOZDRQW', 'balance': 60, 'diff': 6},
 '17_5': GossipMessage:Transaction:{'hash': 'YNRUBQLEQP', 'balance': 76, 'diff': 5},
 '17_7': GossipMessage:Transaction:{'hash': 'GDWCLWLURW', 'balance': 62, 'diff': 9},
 '17_6': GossipMessage:Transaction:{'hash': 'YVIFOYOEPG', 'balanc

In [187]:
sim.peers[15].storage['msg_data'].txs

{'17_1': GossipMessage:Transaction:{'hash': 'QGHTQOOZXC', 'balance': 94, 'diff': 6},
 '17_2': GossipMessage:Transaction:{'hash': 'HOTHJOWJHN', 'balance': 94, 'diff': 5},
 '16_2': GossipMessage:Transaction:{'hash': 'YUGMUCUBPA', 'balance': 83, 'diff': 9},
 '17_3': GossipMessage:Transaction:{'hash': 'CWTXUKBSHY', 'balance': 88, 'diff': 2},
 '16_3': GossipMessage:Transaction:{'hash': 'SCKPYFGFHN', 'balance': 78, 'diff': 5},
 '16_4': GossipMessage:Transaction:{'hash': 'VJOFQYTIZY', 'balance': 76, 'diff': 2},
 '17_5': GossipMessage:Transaction:{'hash': 'YNRUBQLEQP', 'balance': 76, 'diff': 5},
 '17_6': GossipMessage:Transaction:{'hash': 'UNYMWYXDQR', 'balance': 72, 'diff': 7},
 '16_6': GossipMessage:Transaction:{'hash': 'KWDXKAXOTC', 'balance': 66, 'diff': 4},
 '16_7': GossipMessage:Transaction:{'hash': 'RFAVOZDRQW', 'balance': 60, 'diff': 6},
 '17_8': GossipMessage:Transaction:{'hash': 'UJISJGMIZI', 'balance': 67, 'diff': 1},
 '16_5': GossipMessage:Transaction:{'hash': 'EINRRLIBQY', 'balanc

Peers see different versions of the same message!
This is an issue as it might violate integrity guarantees. For example, peers might have a different view on the client's balance. 

How to fix this? 
- One way to solve this is to use a (consensus algorithm)[https://en.wikipedia.org/wiki/Consensus_(computer_science)]


# Consensus algorithm


The consensus is a process that allows achieving a consistent view on a value (agreement). 
Some of the peers may fail or be unreliable, so consensus protocols must be fault-tolerant or resilient. The peers must communicate with one another and agree on a single value.

The consensus problem is fundamental in all distributed systems. One approach to generating consensus is for all processes (agents) to agree on a majority value. In this context, a majority requires at least one more than half of the available votes (where each process is given a vote). However, one or more faulty processes may skew the resultant outcome such that consensus may not be reached or reached incorrectly.



#  Exercise 

In this notebook we ask to implement a consensus service and show that all honest peers accept the same value.  
You can assume that an elected leader is never faulty. 
Here are some poissible algorithms you can consider: 
- **Majority Voting**. Fully connected network. Send votes to all nodes. Choose the value based on majority/super-majority. How many rounds/phases do you need?  
- **Majority Voting with Neighbours**. Connected network. Send votes to all neighbors. Choose the value based on majority/super-majority of your neighbors. How many rounds/phases you need for full convergence? 
- **Consensus through a lottery**. Send transactions through gossip. Everybody runs some lottery mechanism: that both takes time and chooses one or several nodes. This(these) nodes decide which version of the transaction to pick and send through gossip a decided version of transactions (block). How many rounds of lottery you need for convergence? 

**You can choose any of the above algorithm or propose your own**


In [190]:
# Initialize the experiment:
import networkx as nx
import p2psimpy as p2p
import warnings
warnings.filterwarnings('ignore')

# Load the previous experiment configurations
exper = p2p.BaseSimulation.load_experiment(expr_dir='crash_gossip')
#exper = p2p.BaseSimulation.load_experiment(expr_dir='gossip_expr')
#exper = p2p.BaseSimulation.load_experiment(expr_dir='gossip_exprNew')

Locations, topology, peer_services, serv_impl = exper


In [191]:
import random
import string

from p2psimpy import BaseMessage, GossipMessage, MessageProducer, PullGossipService

from p2psimpy.consts import TEMPERED
from p2psimpy.config import Config, Func, Dist

class Transaction(BaseMessage):
    pass 

class ConflictMessageProducer(MessageProducer):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.balance = 100 
        
        
    def _generate_tx(self):        
        msg_phash = ''.join(random.choices(string.ascii_uppercase, k=5))
        diff = random.randint(1, 9)        
        data = {'hash': msg_phash, 'balance': self.balance-diff, 'diff': diff}
        tx = Transaction(self.peer, data)
        
        msg_id = '_'.join((str(self.peer.peer_id), str(self.counter)))
        msg_ttl = self.init_ttl
        return GossipMessage(self.peer, msg_id, tx, msg_ttl, 
            pre_task=self.pre_task, post_task=self.post_task)
    
    def produce_transaction(self):
        # generate new transaction
        msg = self._generate_tx()
        if self.balance - msg.data.data['diff'] < 0:
            return
        
        cons = list(self.peer.connections.keys())
        m_ix = len(cons) // 2
        for p in cons[:m_ix]:
            self.peer.send(p, msg)
            
        # Generate conflicting message as if previous transaction hasn't happened  
        msg = self._generate_tx()
        for p in cons[m_ix:]:
            self.peer.send(p, msg)
        
        self.peer.store('msg_time', str(self.peer.peer_id) +'_' + str(self.counter), self.peer.env.now)
        self.peer.store('msg_data', str(self.peer.peer_id) + '_' + str(self.counter), msg)
        self.balance -= msg.data.data['diff']
        self.counter+=1

def validate_task(msg, peer):
    # time it takes to verify the signature
    crypto_verify = Dist('norm', (1, 0.2)) 
    # time to verify the message data
    msg_verify = Dist("lognorm", (0.49512563, 4.892564, 0.0425785)) 
    
    yield peer.env.timeout(crypto_verify.get() + msg_verify.get())
    if isinstance(msg, Transaction):        
        tx = msg.data
        if tx == TEMPERED or tx['balance'] < 0:
            # You can decide what to do in this case.
            return False
    return True

class MsgConfig(Config):
    pre_task = Func(validate_task)

peer_services['client'].service_map['MessageProducer'] = MsgConfig
serv_impl['MessageProducer'] = ConflictMessageProducer
serv_impl['RangedPullGossipService'] = PullGossipService


In [192]:
peer_services

{'client': PeerType(config=<class 'p2psimpy.config.PeerConfig'>, service_map={'BaseConnectionManager': None, 'MessageProducer': <class '__main__.MsgConfig'>}),
 'peer': PeerType(config=<class 'p2psimpy.config.PeerConfig'>, service_map={'BaseConnectionManager': None, 'RandomDowntime': <class 'p2psimpy.config.DowntimeConfig'>, 'RangedPullGossipService': <class 'p2psimpy.config.GossipConfig'>})}

In [193]:
# Init Graph
sim = p2p.BaseSimulation(Locations, topology, peer_services, serv_impl)
sim.run(2_200)

In [194]:
import pandas as pd

def message_data(sim, peer_id, storage_name):
    store = sim.peers[peer_id].storage[storage_name].txs
    for msg_id, tx in store.items():
        client_id, msg_num = msg_id.split('_')
        client_tx = sim.peers[int(client_id)].storage[storage_name].txs[msg_id]
        yield (int(msg_num), tx.data == client_tx.data)
        
def get_gossip_table(sim, storage_name, func):
    return pd.DataFrame({k: dict(func(sim, k, storage_name)) 
                         for k in set(sim.types_peers['peer'])}).sort_index()

    
df = get_gossip_table(sim, 'msg_data', message_data)
df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,False,True,True,False,False,False,True,True,False,True,NaN,True,True,False,True
2,False,False,NaN,True,False,False,True,False,False,True,NaN,False,False,False,True
3,False,False,True,True,NaN,False,True,False,False,NaN,NaN,False,False,False,True
4,False,NaN,True,False,True,True,False,NaN,False,True,NaN,NaN,NaN,True,False
5,False,NaN,NaN,NaN,True,False,False,False,False,True,NaN,False,NaN,False,True
6,False,NaN,True,NaN,NaN,True,NaN,False,False,True,NaN,False,NaN,True,True


In [195]:
df[df==False].count()

1     6
2     2
3     0
4     2
5     2
6     4
7     2
8     4
9     6
10    0
11    0
12    4
13    2
14    4
15    1
dtype: int64

In [196]:
sim.peers[1].storage['msg_data'].txs

{'17_2': GossipMessage:Transaction:{'hash': 'PYOUC', 'balance': 92, 'diff': 3},
 '17_3': GossipMessage:Transaction:{'hash': 'UUHCM', 'balance': 83, 'diff': 9},
 '17_4': GossipMessage:Transaction:{'hash': 'GIPQL', 'balance': 80, 'diff': 3},
 '17_1': GossipMessage:Transaction:{'hash': 'ABMCE', 'balance': 97, 'diff': 3},
 '17_6': GossipMessage:Transaction:{'hash': 'CRYQO', 'balance': 78, 'diff': 1},
 '16_6': GossipMessage:Transaction:{'hash': 'ONTUU', 'balance': 80, 'diff': 6},
 '16_4': GossipMessage:Transaction:{'hash': 'EDYVA', 'balance': 88, 'diff': 6},
 '16_3': GossipMessage:Transaction:{'hash': 'DSOHH', 'balance': 95, 'diff': 1},
 '16_5': GossipMessage:Transaction:{'hash': 'JEBFZ', 'balance': 84, 'diff': 4},
 '16_2': GossipMessage:Transaction:{'hash': 'SDFTE', 'balance': 89, 'diff': 9},
 '16_1': GossipMessage:Transaction:{'hash': 'OHMCB', 'balance': 91, 'diff': 9}}

In [197]:
sim.peers[2].storage['msg_data'].txs

{'17_1': GossipMessage:Transaction:{'hash': 'WQTZZ', 'balance': 95, 'diff': 5},
 '16_1': GossipMessage:Transaction:{'hash': 'PPFCA', 'balance': 98, 'diff': 2},
 '17_3': GossipMessage:Transaction:{'hash': 'HSZZV', 'balance': 84, 'diff': 8},
 '17_2': GossipMessage:Transaction:{'hash': 'TVMVM', 'balance': 86, 'diff': 9}}

In [198]:
sim.peers[3].storage['msg_data'].txs

{'16_1': GossipMessage:Transaction:{'hash': 'PPFCA', 'balance': 98, 'diff': 2},
 '16_3': GossipMessage:Transaction:{'hash': 'PJADR', 'balance': 94, 'diff': 2},
 '16_4': GossipMessage:Transaction:{'hash': 'PKGJA', 'balance': 88, 'diff': 6},
 '16_6': GossipMessage:Transaction:{'hash': 'FHZQR', 'balance': 79, 'diff': 7}}

In [199]:
sim.peers[4].storage['msg_data'].txs

{'16_1': GossipMessage:Transaction:{'hash': 'PPFCA', 'balance': 98, 'diff': 2},
 '17_1': GossipMessage:Transaction:{'hash': 'ABMCE', 'balance': 97, 'diff': 3},
 '17_2': GossipMessage:Transaction:{'hash': 'TVMVM', 'balance': 86, 'diff': 9},
 '17_3': GossipMessage:Transaction:{'hash': 'UUHCM', 'balance': 83, 'diff': 9},
 '17_4': GossipMessage:Transaction:{'hash': 'PVLJG', 'balance': 79, 'diff': 4},
 '16_2': GossipMessage:Transaction:{'hash': 'VOKMD', 'balance': 96, 'diff': 2}}

In [200]:
sim.peers[5].storage['msg_data'].txs

{'17_1': GossipMessage:Transaction:{'hash': 'ABMCE', 'balance': 97, 'diff': 3},
 '17_2': GossipMessage:Transaction:{'hash': 'TVMVM', 'balance': 86, 'diff': 9},
 '16_4': GossipMessage:Transaction:{'hash': 'PKGJA', 'balance': 88, 'diff': 6},
 '16_5': GossipMessage:Transaction:{'hash': 'HEKEH', 'balance': 86, 'diff': 2}}

In [201]:
sim.peers[6].storage['msg_data'].txs

{'16_1': GossipMessage:Transaction:{'hash': 'OHMCB', 'balance': 91, 'diff': 9},
 '16_3': GossipMessage:Transaction:{'hash': 'DSOHH', 'balance': 95, 'diff': 1},
 '16_5': GossipMessage:Transaction:{'hash': 'JEBFZ', 'balance': 84, 'diff': 4},
 '16_6': GossipMessage:Transaction:{'hash': 'ONTUU', 'balance': 80, 'diff': 6},
 '17_1': GossipMessage:Transaction:{'hash': 'ABMCE', 'balance': 97, 'diff': 3},
 '16_4': GossipMessage:Transaction:{'hash': 'EDYVA', 'balance': 88, 'diff': 6},
 '17_3': GossipMessage:Transaction:{'hash': 'HSZZV', 'balance': 84, 'diff': 8},
 '16_2': GossipMessage:Transaction:{'hash': 'SDFTE', 'balance': 89, 'diff': 9},
 '17_2': GossipMessage:Transaction:{'hash': 'TVMVM', 'balance': 86, 'diff': 9},
 '17_4': GossipMessage:Transaction:{'hash': 'GIPQL', 'balance': 80, 'diff': 3},
 '17_6': GossipMessage:Transaction:{'hash': 'CRYQO', 'balance': 78, 'diff': 1}}

In [202]:
sim.peers[7].storage['msg_data'].txs

{'17_1': GossipMessage:Transaction:{'hash': 'ABMCE', 'balance': 97, 'diff': 3},
 '17_2': GossipMessage:Transaction:{'hash': 'TVMVM', 'balance': 86, 'diff': 9},
 '17_4': GossipMessage:Transaction:{'hash': 'PVLJG', 'balance': 79, 'diff': 4},
 '17_3': GossipMessage:Transaction:{'hash': 'UUHCM', 'balance': 83, 'diff': 9},
 '16_2': GossipMessage:Transaction:{'hash': 'VOKMD', 'balance': 96, 'diff': 2},
 '16_1': GossipMessage:Transaction:{'hash': 'PPFCA', 'balance': 98, 'diff': 2},
 '17_5': GossipMessage:Transaction:{'hash': 'TSQOZ', 'balance': 78, 'diff': 2}}

In [203]:
sim.peers[8].storage['msg_data'].txs

{'17_1': GossipMessage:Transaction:{'hash': 'ABMCE', 'balance': 97, 'diff': 3},
 '17_2': GossipMessage:Transaction:{'hash': 'TVMVM', 'balance': 86, 'diff': 9},
 '17_3': GossipMessage:Transaction:{'hash': 'HSZZV', 'balance': 84, 'diff': 8},
 '17_5': GossipMessage:Transaction:{'hash': 'TSQOZ', 'balance': 78, 'diff': 2},
 '17_6': GossipMessage:Transaction:{'hash': 'KPNQA', 'balance': 75, 'diff': 4},
 '16_1': GossipMessage:Transaction:{'hash': 'PPFCA', 'balance': 98, 'diff': 2}}

In [204]:
sim.peers[9].storage['msg_data'].txs

{'16_1': GossipMessage:Transaction:{'hash': 'OHMCB', 'balance': 91, 'diff': 9},
 '16_2': GossipMessage:Transaction:{'hash': 'SDFTE', 'balance': 89, 'diff': 9},
 '16_4': GossipMessage:Transaction:{'hash': 'EDYVA', 'balance': 88, 'diff': 6},
 '17_1': GossipMessage:Transaction:{'hash': 'ABMCE', 'balance': 97, 'diff': 3},
 '17_3': GossipMessage:Transaction:{'hash': 'HSZZV', 'balance': 84, 'diff': 8},
 '17_2': GossipMessage:Transaction:{'hash': 'TVMVM', 'balance': 86, 'diff': 9},
 '16_6': GossipMessage:Transaction:{'hash': 'ONTUU', 'balance': 80, 'diff': 6},
 '16_3': GossipMessage:Transaction:{'hash': 'DSOHH', 'balance': 95, 'diff': 1},
 '16_5': GossipMessage:Transaction:{'hash': 'JEBFZ', 'balance': 84, 'diff': 4}}

In [205]:
sim.peers[10].storage['msg_data'].txs

{'17_1': GossipMessage:Transaction:{'hash': 'WQTZZ', 'balance': 95, 'diff': 5},
 '17_2': GossipMessage:Transaction:{'hash': 'PYOUC', 'balance': 92, 'diff': 3},
 '17_4': GossipMessage:Transaction:{'hash': 'GIPQL', 'balance': 80, 'diff': 3},
 '17_5': GossipMessage:Transaction:{'hash': 'QLQHW', 'balance': 79, 'diff': 1},
 '17_6': GossipMessage:Transaction:{'hash': 'CRYQO', 'balance': 78, 'diff': 1}}

In [206]:
sim.peers[11].storage['msg_data'].txs

{}

In [207]:
sim.peers[12].storage['msg_data'].txs

{'17_2': GossipMessage:Transaction:{'hash': 'TVMVM', 'balance': 86, 'diff': 9},
 '17_1': GossipMessage:Transaction:{'hash': 'ABMCE', 'balance': 97, 'diff': 3},
 '17_3': GossipMessage:Transaction:{'hash': 'HSZZV', 'balance': 84, 'diff': 8},
 '16_1': GossipMessage:Transaction:{'hash': 'PPFCA', 'balance': 98, 'diff': 2},
 '17_5': GossipMessage:Transaction:{'hash': 'TSQOZ', 'balance': 78, 'diff': 2},
 '17_6': GossipMessage:Transaction:{'hash': 'KPNQA', 'balance': 75, 'diff': 4}}

In [208]:
sim.peers[13].storage['msg_data'].txs

{'17_2': GossipMessage:Transaction:{'hash': 'TVMVM', 'balance': 86, 'diff': 9},
 '17_1': GossipMessage:Transaction:{'hash': 'WQTZZ', 'balance': 95, 'diff': 5},
 '17_3': GossipMessage:Transaction:{'hash': 'HSZZV', 'balance': 84, 'diff': 8},
 '16_1': GossipMessage:Transaction:{'hash': 'PPFCA', 'balance': 98, 'diff': 2}}

In [209]:
sim.peers[14].storage['msg_data'].txs

{'17_3': GossipMessage:Transaction:{'hash': 'UUHCM', 'balance': 83, 'diff': 9},
 '17_2': GossipMessage:Transaction:{'hash': 'PYOUC', 'balance': 92, 'diff': 3},
 '17_1': GossipMessage:Transaction:{'hash': 'ABMCE', 'balance': 97, 'diff': 3},
 '16_6': GossipMessage:Transaction:{'hash': 'ONTUU', 'balance': 80, 'diff': 6},
 '16_4': GossipMessage:Transaction:{'hash': 'EDYVA', 'balance': 88, 'diff': 6},
 '16_3': GossipMessage:Transaction:{'hash': 'DSOHH', 'balance': 95, 'diff': 1},
 '16_5': GossipMessage:Transaction:{'hash': 'JEBFZ', 'balance': 84, 'diff': 4},
 '16_2': GossipMessage:Transaction:{'hash': 'SDFTE', 'balance': 89, 'diff': 9},
 '17_4': GossipMessage:Transaction:{'hash': 'GIPQL', 'balance': 80, 'diff': 3},
 '17_6': GossipMessage:Transaction:{'hash': 'CRYQO', 'balance': 78, 'diff': 1},
 '16_1': GossipMessage:Transaction:{'hash': 'OHMCB', 'balance': 91, 'diff': 9}}

In [210]:
sim.peers[15].storage['msg_data'].txs

{'16_1': GossipMessage:Transaction:{'hash': 'PPFCA', 'balance': 98, 'diff': 2},
 '17_1': GossipMessage:Transaction:{'hash': 'WQTZZ', 'balance': 95, 'diff': 5},
 '16_2': GossipMessage:Transaction:{'hash': 'VOKMD', 'balance': 96, 'diff': 2},
 '17_2': GossipMessage:Transaction:{'hash': 'PYOUC', 'balance': 92, 'diff': 3},
 '17_3': GossipMessage:Transaction:{'hash': 'UUHCM', 'balance': 83, 'diff': 9},
 '17_4': GossipMessage:Transaction:{'hash': 'PVLJG', 'balance': 79, 'diff': 4},
 '16_5': GossipMessage:Transaction:{'hash': 'HEKEH', 'balance': 86, 'diff': 2},
 '17_5': GossipMessage:Transaction:{'hash': 'QLQHW', 'balance': 79, 'diff': 1},
 '17_6': GossipMessage:Transaction:{'hash': 'CRYQO', 'balance': 78, 'diff': 1}}

In [213]:
#I implement the majority voting. Thus, I check all the 15 peers in terms of which version of GossipMessage they have observed from node-16 and node-17. 
#The version that is observed the majority of times per GossipMessage is the one reported in the next cell for both 16 and 17.
#
#First, a few small manipulations. I decreased the k value to k=5 to have smaller message payloads so I can check them more easily
#Then, I also decreased the simulation runtime to sim.run(2_200) so I have less messages to check but at the same time make my point clear.
#
#  15 peers, 2 clients, 6 messages
#after passing the above results from awk-commands and sed-commands of terminal, the messages that win the majority voting are the following

In [214]:
'16_1': GossipMessage:Transaction:{'hash': 'PPFCA', 'balance': 98, 'diff': 2}
'16_2': GossipMessage:Transaction:{'hash': 'SDFTE', 'balance': 89, 'diff': 9}
'16_3': GossipMessage:Transaction:{'hash': 'DSOHH', 'balance': 95, 'diff': 1}            
'16_4': GossipMessage:Transaction:{'hash': 'EDYVA', 'balance': 88, 'diff': 6}
'16_5': GossipMessage:Transaction:{'hash': 'JEBFZ', 'balance': 84, 'diff': 4}
'16_6': GossipMessage:Transaction:{'hash': 'ONTUU', 'balance': 80, 'diff': 6}            
            
'17_1': GossipMessage:Transaction:{'hash': 'ABMCE', 'balance': 97, 'diff': 3}
'17_2': GossipMessage:Transaction:{'hash': 'TVMVM', 'balance': 86, 'diff': 9}
'17_3': GossipMessage:Transaction:{'hash': 'HSZZV', 'balance': 84, 'diff': 8}            
'17_4': GossipMessage:Transaction:{'hash': 'GIPQL', 'balance': 80, 'diff': 3}
'17_5': GossipMessage:Transaction:{'hash': 'TSQOZ', 'balance': 78, 'diff': 2}            
'17_6': GossipMessage:Transaction:{'hash': 'CRYQO', 'balance': 78, 'diff': 1}            
            
            

SyntaxError: invalid syntax (<ipython-input-214-883335b34117>, line 1)